In [1]:
import pandas as pd
import numpy as np

In [87]:

print("Processing Files started. This might take a minute... PLEASE WAIT.")
ken_loe = pd.read_excel("LOE20250219.xlsx", sheet_name=1, engine="openpyxl")
tzs_loe = pd.read_excel("TZSLOE20250219.xlsx", sheet_name=1, engine="openpyxl")
ugx_loe = pd.read_excel("UGXLOE20250219.xlsx", sheet_name=1, engine="openpyxl")
rwf_loe = pd.read_excel("RWFLOE20250219.xlsx", sheet_name=1, engine="openpyxl")
print("\nLoe file (s) read...\nProceeding to outgoing... PLEASE WAIT")


df_outgoing = pd.read_excel("All Outgoing Transaction Details TWI_2025_02_20_080441.xlsx", sheet_name=0, engine="openpyxl", header=1)
print("\nOutgoing file read...\nProceeding to Analysis... PLEASE WAIT")


Processing Files started. This might take a minute... PLEASE WAIT.

Loe file (s) read...
Proceeding to outgoing... PLEASE WAIT

Outgoing file read...
Proceeding to Analysis... PLEASE WAIT


In [88]:
def clean_dataframe(df):
    df = df.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))
    df.columns = df.columns.str.strip()
    return df

ugx_loe = clean_dataframe(ugx_loe)
tzs_loe = clean_dataframe(tzs_loe)
rwf_loe = clean_dataframe(rwf_loe)
ken_loe = clean_dataframe(ken_loe)

In [89]:
#LOE FILTERS

tzs_acquiring = ["USD1403441530033","TZS1403441530033","USD1666600010033","TZS1666600010066"]
ugx_acquiring = ["UGX1403400010022","UGX1666600010022","USD1666600010022","USD1403441530001"]
rwf_acquiring = ["RWF1666600010044","USD1403441530001","USD1666600010066","RWF1403441530001"]
ken_acquiring = ["KES1666600010001","USD1403441530001","USD1666600010001","KES1403441530001"]

tz_mc_loe = tzs_loe[tzs_loe["Debit External"].isin(tzs_acquiring)&tzs_loe["Credit External"].isin(tzs_acquiring)]

rwf_mc_loe = rwf_loe[rwf_loe["Debit External"].isin(rwf_acquiring)&rwf_loe["Credit External"].isin(rwf_acquiring)]

ugx_mc_loe = ugx_loe[ugx_loe["Debit External"].isin(ugx_acquiring)&ugx_loe["Credit External"].isin(ugx_acquiring)]
ken_mc_loe = ken_loe[ken_loe["Debit External"].isin(ken_acquiring)&ken_loe["Credit External"].isin(ken_acquiring)]

ugx_mc_loe.reset_index(drop=True)



#DATA CLEANING FOR OUTGOING

# ✅ Convert Data Types
df_outgoing["ORIGINALAMT"] = pd.to_numeric(df_outgoing["ORIGINALAMT"], errors="coerce").fillna(0.00).abs()
df_outgoing["ORIGTIME"] = pd.to_datetime(df_outgoing["ORIGTIME"], errors="coerce").fillna(pd.Timestamp("1970-01-01"))
df_outgoing["BUS. DAY"] = pd.to_datetime(df_outgoing["BUS. DAY"], errors="coerce").fillna(pd.Timestamp("1970-01-01"))

df_outgoing["EXPTRANID"] = df_outgoing["EXPTRANID"].astype(str).str[:25]  # Keep only first 25 characters

targ_network = [22]
targ_origid = ['KCBK']
targ_date = ['2025-02-19']
mc_acquiring_out = df_outgoing[
        df_outgoing["NETWORK"].isin(targ_network)#& df_outgoing["ORIGID"].isin(targ_origid)#&df_outgoing["BUS. DAY"].astype(str).isin(targ_date)
]


In [90]:
currencies = {
    "tz": tz_mc_loe,
    "ug": ugx_mc_loe,
    "rw": rwf_mc_loe,
    'ke': ken_mc_loe
}

# Ensure mc_acquiring_out is not a slice
mc_acquiring_out = mc_acquiring_out.copy()
mc_acquiring_out.loc[:, 'key'] = mc_acquiring_out['PAN'].astype(str) + mc_acquiring_out['ORIGINALAMT'].astype(str)

for code, df in currencies.items():
    df = df.copy()  # Ensure you're working with a fresh copy
    df.loc[:, 'key'] = df['PAN'].astype(str).str.strip() + df['Debit Amount'].astype(str)
    df.loc[:, 'Status'] = df['key'].isin(mc_acquiring_out['key']).map({True: 'matched', False: 'Non-matched'})
    #save to excel
    df.to_excel(f"{code}_matched_loe.xlsx", index=False)

    # ✅ Print only non-matched rows
    non_matched = df[df['Status'] == 'Non-matched']
    print(f"\n{code.upper()} - Non-matched Rows ({len(non_matched)} found):")
    print(non_matched)



TZ - Non-matched Rows (1 found):
         Date      DocNo No    Debit External Debit Account  Cur D  \
0  19/02/2025  671629846  1  USD1666600010033      30332201    840   

   Debit Amount   Credit External    Credit Account  Cur C  Credit Amount  \
0         703.8  USD1403441530033  USD1403441530033    840          703.8   

  Entry Identifier               PAN Card product Additional Customer info  \
0        OPEXT_P10  5598090012213557                                         

                     key       Status  
0  5598090012213557703.8  Non-matched  

UG - Non-matched Rows (0 found):
Empty DataFrame
Columns: [Date, DocNo, No, Debit External, Debit Account, Cur D, Debit Amount, Credit External, Credit Account, Cur C, Credit Amount, Entry Identifier, PAN, Card product, Additional Customer info, key, Status]
Index: []

RW - Non-matched Rows (1 found):
          Date      DocNo No    Debit External     Debit Account  Cur D  \
19  19/02/2025  671849235  1  RWF1666600010044  RWF140

In [106]:

loes = {
    "tz": tz_mc_loe,
    "ug": ugx_mc_loe,
    "rw": rwf_mc_loe,
    'ken': ken_mc_loe
}


currency_codes = {
    "USD": {"debit": ['USD1403441530001','USD1403441530033'], "credit": ['USD1666600010001','USD1666600010022','USD1666600010033','USD1666600010066']},
    "KES": {"debit": ['KES1403441530001'], "credit": ['KES1666600010001']},
    "RWF": {"debit": ['RWF1403441530001'], "credit": ['RWF1666600010044']},
    "TZS": {"debit": ['TZS1403441530033'], "credit": ['TZS1666600010066']},
    "UGX": {"debit": ['UGX1403400010022'], "credit": ['UGX1666600010022']}
}

results ={}
for curr, acc in currency_codes.items():

    debit_codes = acc['debit']
    credit_codes = acc['credit']
    
    for country, loe in loes.items():

        loe_sale = loe[loe['Debit External'].isin(debit_codes)&loe['Credit External'].isin(credit_codes)]
        loe_rev = loe[loe['Credit External'].isin(debit_codes)&loe['Debit External'].isin(credit_codes)]

        sale_amt = loe_sale['Debit Amount'].sum()
        sale_count = loe_sale.shape[0]

        if sale_count > 0 or sale_amt > 0:
            results[(country, curr, 'SALE')] = {
                'Count': sale_count,
                'Amount': sale_amt
            }

        rev_count = loe_rev['Debit Amount'].count()
        rev_amt = loe_rev['Debit Amount'].sum()

        if rev_count > 0 or rev_amt > 0:
            results[(country, curr, 'REV')] = {
                'Count': rev_count,
                'Amount': rev_amt
            }

        if sale_amt != 0 or rev_amt != 0:
            print(f"{country.upper()} - {curr}: SALE count: {sale_count:,}, Amount: {sale_amt:,.2f} | REV count: {rev_count:,}, Amount: {rev_amt:,.2f}")

TZ - USD: SALE count: 1,637, Amount: 479,781.65 | REV count: 11, Amount: 2,419.63
UG - USD: SALE count: 129, Amount: 39,094.88 | REV count: 1, Amount: 1,180.00
RW - USD: SALE count: 6, Amount: 2,587.55 | REV count: 0, Amount: 0.00
KEN - USD: SALE count: 1,217, Amount: 541,145.70 | REV count: 19, Amount: 16,070.16
KEN - KES: SALE count: 8,371, Amount: 47,497,243.19 | REV count: 61, Amount: 216,874.11
RW - RWF: SALE count: 69, Amount: 4,442,066.00 | REV count: 1, Amount: 200,000.00
TZ - TZS: SALE count: 1,586, Amount: 177,483,868.85 | REV count: 7, Amount: 612,700.00
UG - UGX: SALE count: 952, Amount: 168,449,549.00 | REV count: 2, Amount: 1,900,030.00


In [117]:
rwf_sale = results.get(('rw', 'RWF', 'SALE'), {'Count': 0, 'Amount': 0})
rwf_rev = results.get(('rw', 'RWF', 'REV'), {'Count': 0, 'Amount': 0})

rwf_sale_usd = results.get(('rw', 'USD', 'SALE'), {'Count': 0, 'Amount': 0})
rwf_rev_usd = results.get(('rw', 'USD', 'REV'), {'Count': 0, 'Amount': 0})

tzs_sale = results.get(('tz','TZS', 'SALE'), {'Count': 0, 'Amount': 0})
tzs_rev = results.get(('tz','TZS', 'REV'), {'Count': 0, 'Amount': 0})

tzs_sale_usd = results.get(('tz','USD', 'SALE'), {'Count': 0, 'Amount': 0})
tzs_rev_usd = results.get(('tz','USD', 'REV'), {'Count': 0, 'Amount': 0})

ugx_sale = results.get(('ug','UGX', 'SALE'), {'Count': 0, 'Amount': 0})
ugx_rev = results.get(('ug','UGX', 'REV'), {'Count': 0, 'Amount': 0})

ugx_sale_usd = results.get(('ug','USD', 'SALE'), {'Count': 0, 'Amount': 0})
ugx_rev_usd = results.get(('ug','USD', 'REV'), {'Count': 0, 'Amount': 0})

ken_sale = results.get(('ke','KES', 'SALE'), {'Count': 0, 'Amount': 0})
ken_rev = results.get(('ke','KES', 'REV'), {'Count': 0, 'Amount': 0})

ken_sale_usd = results.get(('ke','USD', 'SALE'), {'Count': 0, 'Amount': 0})
ken_rev_usd = results.get(('ke','USD', 'REV'), {'Count': 0, 'Amount': 0})

print(f"RWF Sale Amount: {tzs_sale_usd['Amount']:,.2f}")
print(f"RWF Rev Amount: {tzs_rev_usd['Amount']:,.2f}")  # Will show 0 if not found

RWF Sale Amount: 479,781.65
RWF Rev Amount: 2,419.63
